In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
library(cowplot)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

# Group proposal on developing predictive model in detecting and preventing forest fires in the Algerian regions.

In the Algerian regions(Northern Africa) of **Bejaia** and **Sidi Bel-abbes** during the period, between June 2012 and September 2012, forest fires ravaged an estimated 20,000 hectares of land. 
Our project aims to construct a prediction model for forest fires and we will use the Algerian Forest Fires Dataset, which contains data from two sub-regions.

**Brief explanation**
* Temperature: Degree Celsius
* RH: Relative Humidity (%)
* Ws: Wind speed (km/h)
* ISI (Initial Spread Index): estimation of the anticipated spread of a fire that is based on FFMC. 
* FFMC (fine fuel moisture code): the moisture content based on litter or other fine fuels. 
* DMC (Duff Moisture Code) and DC (Drought Code): is the average rating of moisture in the organic layers of the forest.
* BUI (Buildup Index): the total amount of fuel that can be used for a combustion reaction, this number is based on the DMC and the DC. 
* FWI (Fire Weather Index): the rating of the strength of the fire.



Tidying dataset: ... ... #There exist two dataset from the source, and we ....!!!

In [7]:
B_forestfire_data <- read_csv("https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv", skip = 1, 
                            col_types = "ncnnnnnnnnnnnf")%>%
                slice(1:124)%>%
                mutate(month = case_when(month == '06' ~ "June",
                                   month == '07' ~ "Jul",
                                   month == '08' ~ "Aug",
                                   month == '09' ~ "Sep"),
                      Region = "Bejaia")%>%
                filter(Classes != 'Classes')

S_forestfire_data <- read_csv("https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv", skip = 126, 
                            col_types = "ncnnnnnnnnnnnf")%>%
                mutate(month = case_when(month == '06' ~ "June",
                                   month == '07' ~ "Jul",
                                   month == '08' ~ "Aug",
                                   month == '09' ~ "Sep"),
                      Region = "Sidi Bel-abbes")%>%
                filter(Classes != 'Classes')

#combining two dataset into one for tidyness
forest_fire_data <- rbind(B_forestfire_data, S_forestfire_data)

head(forest_fire_data, n = 3)
tail(forest_fire_data, n = 3)

Warning message:
“16 parsing failures.
row         col   expected      actual                                                                                                       file
123 day         a number   -           'https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv'
123 NA          14 columns 1 columns   'https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv'
124 day         a number   day         'https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv'
124 year        a number   year        'https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv'
124 Temperature a number   Temperature 'https://raw.githubusercontent.com/zjr-pjt/Proposal-Group-2/main/Algerian_forest_fires_dataset_UPDATE.csv'
... ........... .......... ........... ..............................................

day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
1,June,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,not fire,Bejaia
2,June,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,not fire,Bejaia
3,June,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire,Bejaia


day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
28,Sep,2012,27,87,29,0.5,45.9,3.5,7.9,0.4,3.4,0.2,not fire,Sidi Bel-abbes
29,Sep,2012,24,54,18,0.1,79.7,4.3,15.2,1.7,5.1,0.7,not fire,Sidi Bel-abbes
30,Sep,2012,24,64,15,0.2,67.3,3.8,16.5,1.2,4.8,0.5,not fire,Sidi Bel-abbes


**To tidy the dataset, we gave appropriate column types for the variables, the ‘Classes’ column was filtered out due to some rows of NA data.**

In [4]:
#1
nrow(forest_fire_data)

[1] 243

**!!!We observed from the plots, and then we explain which predictors we found (has influence on the Classes) should to be added to the prediction model... ...****Exploratory Steps**

Firstly,this is a small dataset and we decided to slice the data with 76% for the training set and 24% for the testing set. This will ensure that the training set is large enough while still having sufficient data for the testing set to test the accuracy

We explored the data to see their sizes and the count under each class. We made tables and plots of FWI values evaluating whether a fire class always has a greater FWI value than a non-fire class in two subregions.

We'll use different variables to create plots to visualize the relationships between the vectors.


**!!! We observed from the plots, and then we explain which predictors we found (has influence on the Classes) should to be added to the prediction model... ...**

In [5]:
set.seed(100)

forest_split <- initial_split(forest_fire_data, prop = 0.76, strata = Classes)

training <- training(forest_split)
testing <- testing(forest_split)


In [6]:

df_FWI <- training %>%
            select(FWI, Classes, month) %>%
            group_by(month, Classes)%>%
            summarize(FWI = mean(FWI)) %>%
            mutate(month = factor(month, levels = c("June", "Jul", "Aug", "Sep")))

df_FWI_plot <- d_FWI %>%
                    ggplot(aes(x = month, y = FWI, fill = Classes))+
                    geom_bar(stat = "identity")+
                    facet_grid(Classes~.)+
                    labs(x = "Month", y = "FWI Index", fill = "Classes")+
                    theme(text = element_text(size = 18))

df_FWI_plot
options(repr.plot.width = 10, repr.plot.height = 6)
            

`summarise()` regrouping output by 'month' (override with `.groups` argument)



ERROR: Error in eval(lhs, parent, parent): object 'd_FWI' not found


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)

temp_ws <- training %>%
            ggplot(aes(x = Temperature, y = RH, color = Classes))+
            geom_point(size = 2, alpha = 0.5)+
            labs(x = "Temperature",y = "Relative Humidity", color = "Classes")+
            theme(text = element_text(size = 20))
temp_ws


**Both plots showed a negative moderately strong linear relationship. Fires become more likely to occur as temperatures increase and relative humidity decreases.Fires predominantly occur when temperatures are above thirty degrees Celsius and relative humidity is less than 70.**

**Methods**

* Further tuning the classifiers using cross-validation with C = 5, which would prevent the data from under fitting or overfitting, and each observation will only be used once in a validation set.
* Picking a K value by visualizing the Accuracy Estimation vs. Number of Neighbours for the classification models.
* Developing two models: one with only FWI as predictor and the other with ... ... . Testing and comparing their accuracies on testing datasets.

**we also need to conclude the visualization for the classification result , referring to @piazza 679**

**Expected outcomes and significance**

We expect to find that the model could predict future forest fires with an accuracy above 82%. This result can be applied to proactive methods to prevent and detect fire signals. 

We can think of further inquiries such as, does the model fit for other regions in this country, and what about other countries?


!!!observed that set.seed() also had influence over K-value, when set.seed is ..., the best K is ..., and when set.seed() is ..., the best K changes to ...
This could be a further exploratory step to .... ....

**Creating Recipes and Specification**


In [ ]:
#models and recipes for both regions. 
set.seed(100)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
  set_engine("kknn") %>%
  set_mode("classification")

#FWI recipe
recipe_FWI <- recipe(Classes ~ FWI, data = training) %>%
    step_center(all_predictors()) %>%
    step_scale(all_predictors())

#Four vectors recipe
recipe_4v <- recipe(Classes ~ Temperature + RH + Ws + Rain, data = scld_training) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())


**Cross-Validation for two datasets**

In [ ]:
set.seed(30)

fires_fold <- vfold_cv(scld_training, v = 5, strata = Classes)

**Workflow analysis with recipes and models while trying for 10 values of $k$**

In [ ]:
set.seed(30)
#FWI recipe
knn_results_FWI<- workflow() %>%
    add_recipe(recipe_FWI)%>%
    add_model(knn_spec)%>%
    tune_grid(resamples = fires_fold, grid= 10) %>%
    collect_metrics()

#Four vectors recipe
knn_results_4v <- workflow() %>%
    add_recipe(recipe_4v)%>%
    add_model(knn_spec)%>%
    tune_grid(resamples = fires_fold, grid= 10) %>%
    collect_metrics()

In [ ]:
accuracies_FWI <- knn_results_FWI %>%
                 filter(.metric == 'accuracy')


accuracies_4v <- knn_results_4v %>%
                filter(.metric == 'accuracy')


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)
#K Vs. Accuracy plots for both region(FWI recipe)
FWI_K <- ggplot(accuracies_FWI, aes(x = neighbors , y = mean))+
    geom_point()+
    geom_line()+
    labs(x = "Neighbors", y = "Accuracy Estimate")+
    scale_x_continuous(breaks = seq(0,14, by = 1))+
    theme(text = element_text(size = 14))+
    ggtitle("K Vs. Accuracy - FWI recipe")

fv_K <- ggplot(accuracies_4v, aes(x = neighbors , y = mean))+
    geom_point()+
    geom_line()+
    labs(x = "Neighbors", y = "Accuracy Estimate")+
    scale_x_continuous(breaks = seq(0,14, by = 1))+
    theme(text = element_text(size = 14))+
    ggtitle("K Vs. Accuracy - Four Vectors recipe")

plot_grid(FWI_K,fv_K, ncol = 2)

**Best K values**
* K = 10 for the left model
* K = 8 for the right model

In [ ]:
FWI_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) %>%
        set_engine("kknn")%>%
        set_mode("classification")

fv_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 8)%>%
        set_engine("kknn")%>%
        set_mode("classification")

In [ ]:

FWI_fit <- workflow()%>%
        add_recipe(recipe_FWI)%>%
        add_model(FWI_spec)%>%
        fit(data = training)

fv_fit <- workflow()%>%
        add_recipe(recipe_4v)%>%
        add_model(fv_spec)%>%
        fit(data = training)

In [ ]:
FWI_prediction <- predict(FWI_fit, testing)%>%
        bind_cols(testing)

fv_prediction <- predict(fv_fit, testing)%>%
        bind_cols(testing)


In [ ]:

FWI_metrics <- FWI_prediction %>%
            metrics(truth = Classes, estimate = .pred_class)
FWI_metrics


fv_metrics <- fv_prediction %>%
            metrics(truth = Classes, estimate = .pred_class)
fv_metrics